# Curso: Bioestatística — Análise de Covariância
## Autores: Sandro da Silva Camargo e Fernando Cardoso

**Problema**: Testar a resposta de órgãos de animais à adição de hormônios.
O abate por anestesia e usando a decapitação podem provocar respostas diferentes;
Experimento feito em 2 grupos de ratos (mesma idade e sexo): 10 animais mortos por anestesia e 10 decapitados. Os corações foram colocados no soro e a força de contração medida (Y);
* Y: Força de contração
* $X_1$: Tratamento (Anestesia ou Decapitação)
* $X_2$: Co-fator (Peso)

A base de dados está disponível [aqui](https://github.com/Sandrocamargo/biostatistics/blob/master/datasets/ancova-ratos.txt).

Abra este código no seu google colab [clicando aqui](https://colab.research.google.com/github/Sandrocamargo/biostatistics/blob/master/python/bioe_07_Ancova.ipynb).

* A ANCOVA (_Analysis of Covariance_) combina princípios da ANOVA (Análise de Variância) e da Regressão Linear.
* Ela permite comparar as médias de um ou mais grupos (como na ANOVA), ajustando os resultados pelo efeito de uma ou mais covariáveis, isto é, variáveis contínuas que possam influenciar a variável dependente.
* Em outras palavras, a ANCOVA "remove" o efeito de uma covariável para que a comparação entre grupos seja mais justa e precisa.

# Carga de pacotes

In [ ]:
!pip install pingouin
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
import statsmodels.api as sm
import statsmodels.formula.api as smf
import pingouin as pg

# Carga e inspeção dos dados

In [ ]:
dados = pd.read_csv("https://raw.githubusercontent.com/Sandrocamargo/biostatistics/refs/heads/master/datasets/ancova-ratos.txt", sep=" ", decimal=",")
dados['Trt'] = dados['Trt'].astype('category')
dados.info()
dados.head()

# Análise exploratória

In [ ]:
# Correlação entre y e Peso
corr = dados['y'].corr(dados['Peso'])
print(f"Correlação entre y e Peso: {corr:.4f}")

modelo = smf.ols("y ~ Peso", data=dados).fit()
sns.scatterplot(x='Peso', y='y', data=dados)
plt.title(f"Correlação: {corr:.4f}")
sns.lineplot(x=dados['Peso'], y=modelo.fittedvalues, color='red')
plt.xlabel("Peso (kg)")
plt.ylabel("Força de Contração (Y)")
plt.show()

print(modelo.summary())

**📘 Qualidade geral do modelo**
|Indicador	|Valor	|Interpretação|
|:--|:--|:--|
|R-squared |= 0.796	|O modelo explica 79,6% da variação da variável dependente y. Isso é um valor alto, indicando um bom ajuste.	|
|Adj. R-squared |= 0.785	|Corrige o R² pelo número de parâmetros — ainda muito bom, reforçando que o modelo é consistente.	|
|F-statistic |= 70.30 (p < 0.001)	|Testa a hipótese de que o modelo (com a variável Peso) explica significativamente a variação em y. O valor de p < 0.001 indica que o modelo é altamente significativo.	|

**⚙️ Coeficientes do modelo**
|Termo	|Coef.	|Interpretação|
|:--|:--|:--|
|Intercept |= 1.8359	|Quando o Peso = 0, o valor esperado de y é 1.8359. (Interpretar apenas se fizer sentido no contexto. Se “Peso = 0” não é possível, o intercepto é apenas o ponto onde a reta intercepta o eixo y.)	|
|Peso |= 2.0466	|Cada unidade adicional em Peso está associada a um aumento médio de 2.05 unidades em y, mantendo tudo o mais constante. O efeito é altamente significativo (p < 0.001).	|

**Intervalo de confiança (95%):**
* Peso: [1.534, 2.559] → o verdadeiro valor do coeficiente tem 95% de probabilidade de estar nesse intervalo.
* O intervalo não inclui zero, reforçando que o efeito de Peso é real e estatisticamente significativo.

**🧪 Diagnósticos do modelo**
|Estatística	|Valor	|Interpretação|
|:--|:--|:--|
|Omnibus |= 0.874, Prob = 0.646|	Não há evidência de que os resíduos violem a normalidade.	|
|Jarque-Bera |= 0.088, Prob = 0.957	|Confirma a normalidade dos resíduos.	|
|Durbin-Watson |= 2.079|	Indica ausência de autocorrelação entre resíduos (valores próximos de 2 são ideais).	|
|Cond. No. |= 17.4	|Valor baixo → não há multicolinearidade preocupante (em modelos com uma variável, isso já era esperado).	|

**🧾 Síntese interpretativa**

* ✅ O modelo é estatisticamente significativo e explica cerca de 80% da variabilidade da variável dependente y.
* ✅ A variável Peso tem forte influência positiva e significativa sobre y.
* ✅ Os resíduos atendem aos principais pressupostos (normalidade, independência, homocedasticidade presumida).

**📊 Resumo interpretativo**

O modelo de regressão linear indicou que o peso tem efeito significativo sobre y (β = 2.05; p < 0.001), explicando aproximadamente 80% da variação observada. O diagnóstico de resíduos não revelou violações dos pressupostos de normalidade ou independência (Durbin-Watson = 2.08; p(JB) = 0.957).

In [ ]:
# Boxplots
sns.boxplot(x='Trt', y='y', data=dados)
plt.title("Boxplot da Força de Contração (Y) por Tratamento")
plt.show()

sns.boxplot(x='Trt', y='Peso', data=dados)
plt.title("Boxplot do Peso por Tratamento")
plt.show()

# Estatísticas descritivas
print("\nResumo de Y por grupo:")
print(dados.groupby('Trt', observed=False)['y'].describe())
print("\nResumo de Peso por grupo:")
print(dados.groupby('Trt', observed=False)['Peso'].describe())

**📊 Variável Y**
|Grupo	|n	|Média Desvio padrão	|Interpretação|
|:--|:--|:--|:--|
|Anestesiado	|10	|4,67 ± 0,33|	Resultados mais altos, pouca variabilidade.	|
|Decapitado	|10	|4,28 ± 0,29|	Resultados menores, também com baixa variabilidade.	|

* ➡️ Diferença de médias ≈ 0,39 unidades, com o grupo Anestesiado apresentando valores maiores de Y.
* ➡️ As distribuições parecem relativamente homogêneas (desvios padrão próximos).

**Interpretação:**
O grupo Anestesiado apresentou valores significativamente maiores em média, o que pode indicar efeito do tratamento (a confirmar com teste estatístico, como ANOVA ou t-test).

**⚖️ Variável Peso (covariável)**
|Grupo	|n	|Média	Desvio padrão|	Interpretação|
|:--|:--|:--|:--|
|Anestesiado	|10	|1,410 ± 0,12	|Peso médio mais alto.	|
|Decapitado	|10	|1,169 ± 0,075	|Peso médio menor e menos variável.	|

* ➡️ Diferença de médias ≈ 0,24 unidades. O grupo Anestesiado tem indivíduos visivelmente mais pesados.

**Interpretação:**
* Se o peso influencia Y, essa diferença entre grupos pode confundir a comparação direta entre médias de Y.
* Nessa situação, o ideal é ajustar Y pelo peso (por exemplo, realizando uma análise de covariância (ANCOVA)) para avaliar se o efeito do tratamento (Trt) sobre Y permanece significativo após controlar o efeito do peso.

**🧾 Síntese interpretativa**

* Há diferença média entre tratamentos tanto em Y quanto em Peso.
* O grupo Anestesiado apresenta valores maiores em ambas as variáveis.
* Para determinar se a diferença em Y é devida ao tratamento ou ao peso, deve-se aplicar uma ANCOVA (Y ~ Trt + Peso).

# Análise de covariância

In [ ]:
ancova = pg.ancova(data=dados, dv='y', covar='Peso', between='Trt')
print("\n=== Resultado da ANCOVA ===")
print(ancova)

**Tratamento (Trt)**
* F = 6.46 e p = 0.021 → o efeito do tratamento é estatisticamente significativo (p < 0.05).
* Isso significa que, mesmo após controlar o efeito do peso, ainda há diferença significativa entre os grupos de tratamento quanto à variável dependente (ex: força de contração).
* O eta quadrado parcial (η²p = 0.275) indica um efeito de tamanho moderado: aproximadamente 27,5% da variação ajustada pode ser atribuída ao tratamento.

**Covariável (Peso)**
* F = 62.75 e p < 0.000001 → o peso tem forte efeito sobre a variável dependente.
* O eta quadrado parcial (η²p = 0.787) indica um efeito muito grande: cerca de 78,7% da variação ajustada está associada à covariável.

**Resíduos**
* Representam a variação não explicada pelo modelo.
* Essa soma de quadrados (SS = 0.366) é relativamente pequena, o que reforça que o modelo explica bem os dados.

**📈 Conclusão geral**
* O peso tem forte influência sobre a variável resposta.
* Mesmo controlando o peso, o tratamento ainda exerce um efeito significativo.
* O modelo como um todo explica grande parte da variabilidade dos dados.

# Ajuste de médias

In [ ]:
# Calcula médias ajustadas via modelo linear
modelo_ancova = smf.ols('y ~ Peso + Trt', data=dados).fit()
print("\n=== Modelo ANCOVA ===")
print(modelo_ancova.summary())

# Médias ajustadas por tratamento
em_means = dados.copy()
em_means['ajustado'] = modelo_ancova.fittedvalues
print("\nMédias ajustadas por tratamento:")
print(em_means.groupby('Trt', observed=False)['ajustado'].mean())

**Qual o objetivo da análise?**
* A ANCOVA busca verificar se há diferença entre os grupos de tratamento na variável y, controlando o efeito do peso.
* Em outras palavras: queremos saber se o tratamento afeta y independentemente do peso dos animais.

**📊 Qualidade geral do modelo**
* R² = 0.852 → o modelo explica 85,2% da variação total em y, o que é excelente.
* R² ajustado = 0.835 → mesmo considerando o número de preditores, o modelo mantém boa capacidade explicativa.
* F(2,17) = 49.03, p = 8.72×10⁻⁸ → o modelo global é altamente significativo, ou seja, peso e tratamento, em conjunto, explicam de forma robusta a variação em y.

**⚙️ Interpretação dos coeficientes**
|Termo	|Coef.	|Interpretação|
|:--|:--|:--|
|Intercepto |= 0.8124	|Valor esperado de y para o grupo Anestesiado, quando Peso = 0 (interpretação matemática, não biológica).	|
|Trt[T.Decapitado] = 0.2693, |p = 0.021	|O grupo Decapitado apresenta, em média, +0.27 unidades maiores de y do que o grupo Anestesiado, ajustando o efeito do peso. Como p < 0.05, essa diferença é estatisticamente significativa.	|
|Peso = 2.7359, |p < 0.001	|A cada aumento de 1 unidade de peso, há um acréscimo médio de 2.74 unidades em y, mantendo o tratamento constante. O efeito do peso é forte e altamente significativo.	|

**⚖️ Médias ajustadas (ANCOVA)**
* As médias ajustadas representam os valores de y esperados para cada grupo de tratamento, corrigidos pelo efeito do peso.

|Tratamento	|Média ajustada de y|
|:--|:--|
|Anestesiado	|4.67|
|Decapitado	|4.28|

* ➡️ Embora o coeficiente positivo de “Decapitado” sugira maior valor inicial, após o ajuste pela covariável (peso), o grupo Anestesiado apresenta média ligeiramente maior (4.67 vs 4.28).

* Isso indica que parte da diferença bruta entre grupos era explicada pelo peso  e, após o controle, a diferença se reduz.

**📈 Pressupostos**
* Durbin-Watson = 2.19 → ausência de autocorrelação dos resíduos (bom sinal).
* Teste de normalidade (Omnibus, JB): p > 0.05 → resíduos aproximadamente normais.
* Cond. No. = 31.5 → sem problemas sérios de multicolinearidade.

**🧩 Conclusão**
* O peso exerce forte influência sobre y.
* O tratamento ainda tem efeito significativo mesmo após controlar o peso (p = 0.021).
* O modelo ajustado explica muito bem os dados.
* As médias ajustadas indicam pequenas diferenças entre os grupos, o que reforça a utilidade da ANCOVA: separar o efeito do tratamento do efeito da covariável.

# Teste de pressupostos

In [ ]:
# Normalidade dos resíduos
residuos = modelo_ancova.resid
print("\nShapiro-Wilk para resíduos:")
print(stats.shapiro(residuos))

sns.histplot(residuos, kde=True)
plt.title("Histograma dos Resíduos")
plt.xlabel("Resíduos")
plt.show()

# Homogeneidade de variâncias
print("\nBartlett test:")
print(stats.bartlett(*[dados.loc[dados.Trt == g, 'y'] for g in dados.Trt.unique()]))

print("\nLevene test:")
print(stats.levene(*[dados.loc[dados.Trt == g, 'y'] for g in dados.Trt.unique()]))

sns.boxplot(x='Trt', y=residuos, data=dados)
plt.title("Homogeneidade de variâncias dos resíduos por tratamento")
plt.show()

# Interação (homogeneidade dos coeficientes angulares)

In [ ]:
modelo_interacao = smf.ols('y ~ Trt * Peso', data=dados).fit()
anova_interacao = sm.stats.anova_lm(modelo_interacao, typ=3)
print("\n=== Teste de interação Tratamento*Peso ===")
print(anova_interacao)

**Estrutura geral**

O modelo testou o efeito do Tratamento (Trt), da covariável Peso e da interação entre ambos (Trt:Peso) sobre a variável dependente y.
O objetivo é verificar:
* se o tratamento tem efeito sobre y,
* se o peso influencia y,
* e se o efeito do tratamento depende do peso (interação).

**Interpretação dos resultados**
|Fonte|	F	|p (PR(>F))	|Interpretação|
|:--|:--|:--|:--|
|Intercept	|3.14	|0.095	|O intercepto não é estatisticamente diferente de zero (sem relevância prática isolada).|
|Trt	|0.19	|0.665	|O tratamento não tem efeito significativo sobre y após controlar pelo peso.|
|Peso	|38.86	|< 0.001	|O peso tem forte efeito significativo sobre y. À medida que o peso aumenta, y também tende a aumentar.|
|Trt:Peso|	0.52	|0.480	|A interação não é significativa. O efeito do peso sobre y é semelhante entre os tratamentos.|
|Residual	|—	|—	|Variabilidade não explicada pelo modelo.|

**Conclusão prática**
* O peso é a principal variável associada a y (efeito forte e significativo).
* O tratamento, por si só, não altera significativamente o valor de y.
* A relação entre peso e y é paralela entre os grupos (sem interação significativa).
* Portanto, um modelo sem interação (modelo aditivo) é mais parcimonioso e apropriado.

# Regressões separadas por grupo

In [ ]:
plt.figure()
cores = sns.color_palette("Set2", n_colors=len(dados.Trt.cat.categories))
for i, trt in enumerate(dados.Trt.cat.categories):
    subset = dados[dados.Trt == trt]
    sns.scatterplot(x='Peso', y='y', data=subset, color=cores[i], label=trt)
    modelo = smf.ols('y ~ Peso', data=subset).fit()
    plt.plot(subset['Peso'], modelo.fittedvalues, color=cores[i])
plt.legend()
plt.title("Regressões lineares por tratamento")
plt.xlabel("Peso (kg)")
plt.ylabel("Força de Contração (Y)")
plt.show()